In [1]:
#TODO 1: standardize how to handle dates.
#TODO 2: makes some of the map logic less hacky


from __future__ import print_function
import pandas as pd
import requests
import io
import altair as alt
import functools
from altair import datum
import us
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from vega_datasets import data
import warnings
import io

warnings.filterwarnings('ignore')

In [2]:
# State Level Data
#
# County Level Data
#
# City Level Data

@functools.lru_cache
def fetch_data():
    url_states = "https://covidtracking.com/api/v1/states/daily.json"
    return  pd.read_json(requests.get(url_states).content.decode('utf8'))

@functools.lru_cache
def fetch_state_data(state):
    url_states = f"https://covidtracking.com/api/v1/states/{state}/daily.json"
    return requests.get(url_states).content.decode('utf8')

@functools.lru_cache
def county_data():
    url = "https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv"
    return pd.read_csv(io.StringIO(requests.get(url).content.decode('utf8')))

@functools.lru_cache
def county_stats():
    url = "https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv"
    return pd.read_csv(io.StringIO(requests.get(url).content.decode('utf8')))

In [3]:
class CovidApi:
    def __init__(self):
        self.underlying = fetch_data()
        self.underlying["date"] = self.underlying["date"].astype("str").astype("datetime64")
        self.county_data = county_data()
        self.county_stats = county_stats()
    
    @property
    def states(self):
        return set(self.underlying["state"])
    
    @property
    def props(self):
        return set(self.underlying.columns)
    
    def filter_by(self, state = None, start = None, end = None):
        """
        Arg:
            date: format is `yyyymmdd`
            state: format is two letter state abbreviations. Call `.states` for full list.
        
        Returns:
            Filtered DataFrame.
        """
        data = self.underlying
        assert not state or state in self.states, f"state must be on of {self.states}"
        if state:
            data = data[data["state"] == state]
            
        assert end and start or not end, "If you pass end, you pass must start."
        if start and end:
            date_range = pd.date_range(start=start, end=end)
            data = data[data["date"].isin(date_range)]
        elif start:
            data = data[data["date"] == start]
        return data
    
    def plot_by(self, state, prop, start=None, end=None, color = "blue", plot_type="bar", points=False):
        """
        Arg:
            state: format is two letter state abbreviations. Call `.states` for full list.
            prop: Which property you want to plot over time. Call `.props` for full list.
            color: Chart color
            plot_type: bar or line
        
        Returns:
            Plot.
        """
        if state:
            data = self.filter_by(state=state, start=start, end=end)
        else:
            data = self.underlying.groupby("date").sum().reset_index()
        if plot_type == "bar":
            return alt.Chart(data).mark_bar(color=color).encode(
                x='date',
                y=prop,
                tooltip=["date:T", f"{prop}:Q"]
            )
        else:
            return alt.Chart(data).mark_line(color=color, point=points).encode(
                x='date',
                y=prop,
                tooltip=["date:T", f"{prop}:Q"]
            )
    
    def compare(self, state, first_prop, second_prop, start=None, end=None, plot_type = "line", points = False):
        p1 = self.plot_by(state=state, start=start, end=end, prop=first_prop, color="blue", plot_type=plot_type, points=points)
        p2 = self.plot_by(state=state, start=start, end=end, prop=second_prop, color="red", plot_type=plot_type, points=points)
        return p1 + p2
    
    def compare_states(self, first_state, second_state, prop, start=None, end=None, plot_type = "line", points = False):
        """
        Arg:
            first_state: a state to compare
            second_state: a state to compare
            prop: a dimension to compare between states
        """
        p1 = self.plot_by(state=first_state, start=start, end=end, prop=prop, color="blue", plot_type=plot_type, points=points)
        p2 = self.plot_by(state=second_state, start=start, end=end, prop=prop, color="red", plot_type=plot_type, points=points)
        return p1 + p2
    
    def geo_map(self, date, prop):
        """
        Arg:
           date: format is `yyyymmdd`
           prop: Which property you want to plot over time. Call `.props` for full list.
        
        Returns:
            Plot by state.
        """
        states = alt.topo_feature(data.us_10m.url, 'states')
        source = self.filter_by(start=date)
        source["id"] = source.state.apply(lambda state:us.fips.get(state,0))
        return alt.Chart(states).mark_geoshape().encode(
            color=f'{prop}:Q',
            tooltip=['id:N',f'{prop}:Q']
        ).transform_lookup(
            lookup='id',
            from_=alt.LookupData(data=source, key='id', fields=[prop])
        ).project(
            type='albersUsa'
        ).properties(
            width=500,
            height=300
        )
    
    def geo_map_county(self, date):
        """
        Arg:
            date: format mm/dd/yy
        
        Returns:
            Plot of daily increase by county.
        """
        # TODO: make this less hacky
        date = f"{date}_diff"

        county_combined = self.county_stats.merge(self.county_data)

        dates = [c for c in county_combined.columns if c not in ['countyFIPS', 'County Name', 'State', 'stateFIPS', 'population']]
        for d in dates:
            if len(dates) != dates.index(d) + 1:
                next_c = dates[dates.index(d) + 1]
                county_combined[f"{d}_diff"] = county_combined[next_c] - county_combined[d]

        july_last = county_combined[["countyFIPS", "County Name", "State", "stateFIPS", date]]
        july_last["id"] = july_last["countyFIPS"]

        counties = alt.topo_feature(data.us_10m.url, 'counties')
        return alt.Chart(counties).mark_geoshape().encode(
            color=f'{date}:Q'
        ).transform_lookup(
            lookup='id',
            from_=alt.LookupData(data=july_last, key='id', fields=[date])
        ).encode(
            tooltip=['id:N',f'{date}:Q']
        ).project(
            type='albersUsa'
        ).properties(
            width=500,
            height=300
        )

In [4]:
covid_api = CovidApi()

In [5]:
covid_api.compare(state="NY", first_prop="positiveIncrease", second_prop="death")

alt.LayerChart(...)

In [6]:
covid_api.compare(state="NY", start="20200301", end="20200412", first_prop="positiveIncrease", second_prop="death", points=True)

alt.LayerChart(...)

In [7]:
covid_api.compare_states(first_state="FL", second_state="NY", prop="positiveIncrease")

alt.LayerChart(...)

In [8]:
covid_api.compare(state="FL", start="20200620", end="20200704", first_prop="positiveIncrease", second_prop="deathIncrease", points=True)

alt.LayerChart(...)

In [9]:
covid_api.geo_map(date="20200704", prop="positiveIncrease")

alt.Chart(...)

In [32]:
covid_api.geo_map_county(date="6/30/20")

alt.Chart(...)